In [1]:
import pickle

In [4]:

import torch
import re
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext
import numpy as np
import matplotlib.pyplot as plt
import random
import io
import os
import torchtext

In [5]:
class T_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True) #LSTM
        self.fc = nn.Linear(hidden_size, num_classes)  #przeksztalcenie liniowe
    
    def forward(self, x):
        x = x.unsqueeze(2)
        x = torch.tensor(x)
        x = torch.tensor(x, dtype=torch.float)
        h0 = torch.zeros(1, x.size(0), self.hidden_size) #początkowy  h0
        c0 = torch.zeros(1, x.size(0), self.hidden_size) #początkowy c0
        out, _ = self.lstm(x, (h0, c0))  #LSTM
        out = self.fc(out[:, -1, :]) #przeksztlcam jeszcze liniowo ostatni output
        return out


In [6]:
model_3nuc = pickle.load(open("gram_przewidywanie/3_nuc_model_batch32_hidden25", 'rb'))

In [7]:
model_3nuc

T_LSTM(
  (lstm): LSTM(1, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=2, bias=True)
)

In [9]:
with open('gram_przewidywanie/gram_minus_ess_new.txt','r') as f:
    e_file = f.read()

with open('gram_przewidywanie/gram_minus_ness_new.txt','r') as f1:
    n_file = f1.read()
    
x = re.split('>(DEG\d+)\n([ATGC\n]+)', e_file)
v = [i.replace('\n','') for i in x]

x2 = re.split('>(DNEG\d+)\n([ATGC\n]+)', n_file)
v2 = [i.replace('\n','') for i in x2]

e_names = []
e_seq = []

for i in v:
    match1= re.search('^[ATGC]+$',i)
    match2= re.search('>?DEG',i)
    if match2:
        if not 'available' in i:
            e_names.append(i)
    else:
        if match1:
            e_seq.append(i)

n_names = []
n_seq = []

for i in v2:
    match1= re.search('^[ATGC]+$',i)
    match2= re.search('>?DNEG',i)
    if match2:
        if not 'available' in i:
            n_names.append(i)
    else:
        if match1:
            n_seq.append(i)

def seq_with_label(seq, label):
    
    words_labels = []
    
    for s in seq:
            
        words_labels.append([label, s])
            
    return words_labels

e_data = seq_with_label(e_seq, 'e')
ne_data = seq_with_label(n_seq, 'n')


In [10]:
len(e_data)

13266

In [11]:
len(ne_data)

99777

In [12]:
e_len = len(e_data)

data = e_data + ne_data[:e_len]

random.shuffle(data)

In [42]:
len(data)

26532

In [47]:
data_filtered = [[label, s] for [label, s] in data if label != '' and s != '']

In [48]:
len(data_filtered)

26532

In [62]:
data_filtered[0]

['n',
 'ATGAGAGAGGCTATCATTAAAAGAGCGGCAAAGGAATTGAAAGAGGGCATGTATGTGAATTTAGGGATAGGCTTGCCCACGCTTGTGGCTAATGAAGTGAGCGGGATGAATATCGTTTTCCAAAGCGAGAACGGGCTGTTAGGGATTGGCGCTTACCCTTTAGAGGGGAGCGTTGATGCGGATCTTATCAACGCAGGAAAGGAAACCATAACCGTGGTGCCGGGCGCTTCGTTTTTCAATAGCGCGGATTCGTTTGCGATGATTCGTGGGGGGCATATTGATTTAGCGATTTTAGGGGGGATGGAAGTCTCACAAAATGGGGATTTGGCTAATTGGATGATCCCTAAAAAGCTCATAAAGGGCATGGGAGGGGCTATGGATTTGGTGCATGGCGCTAAAAAAGTGATTGTGATCATGGAGCATTGCAACAAATACGGGGAGTCTAAAGTGAAAAAGGAATGCTCATTGCCCTTAACAGGAAAAGGCGTGGTGCATCAATTGATAACGGATTTAGCGGTGTTTGAGTTTTCCAATAACGCCATGAAATTAGTGGAATTGCAAGAGGGGGTCAGCCTTGATCAAGTGAAAGAAAAGACAGAAGCTGAATTTGAGGTGCGCTTATAG']

In [49]:
train_l = []
valid_l = []
test_l = []


for i, batch in enumerate(data_filtered):
    if i % 5 < 3:
        train_l.append(batch) # 60% danych treningowych
    elif i % 5 == 3: #czyli 3
        valid_l.append(batch) # 20% danych do walidacji
    else:            #czyli gddy i%5 jest 4
        test_l.append(batch) # 20% danych testowych


In [50]:
with open('train.tsv', 'wt') as file:
    tsv_writer = csv.writer(file, delimiter='\t')
    for elem in train_l:
      tsv_writer.writerow(elem)

with open('valid.tsv', 'wt') as file:
    tsv_writer = csv.writer(file, delimiter='\t')
    for elem in valid_l:
      tsv_writer.writerow(elem)

with open('test.tsv', 'wt') as file:
    tsv_writer = csv.writer(file, delimiter='\t')
    for elem in test_l:
      tsv_writer.writerow(elem)

In [59]:
train[0].seq

['ATG',
 'TGA',
 'GAG',
 'AGA',
 'GAG',
 'AGA',
 'GAG',
 'AGG',
 'GGC',
 'GCT',
 'CTA',
 'TAT',
 'ATC',
 'TCA',
 'CAT',
 'ATT',
 'TTA',
 'TAA',
 'AAA',
 'AAA',
 'AAG',
 'AGA',
 'GAG',
 'AGC',
 'GCG',
 'CGG',
 'GGC',
 'GCA',
 'CAA',
 'AAA',
 'AAG',
 'AGG',
 'GGA',
 'GAA',
 'AAT',
 'ATT',
 'TTG',
 'TGA',
 'GAA',
 'AAA',
 'AAG',
 'AGA',
 'GAG',
 'AGG',
 'GGG',
 'GGC',
 'GCA',
 'CAT',
 'ATG',
 'TGT',
 'GTA',
 'TAT',
 'ATG',
 'TGT',
 'GTG',
 'TGA',
 'GAA',
 'AAT',
 'ATT',
 'TTT',
 'TTA',
 'TAG',
 'AGG',
 'GGG',
 'GGA',
 'GAT',
 'ATA',
 'TAG',
 'AGG',
 'GGC',
 'GCT',
 'CTT',
 'TTG',
 'TGC',
 'GCC',
 'CCC',
 'CCA',
 'CAC',
 'ACG',
 'CGC',
 'GCT',
 'CTT',
 'TTG',
 'TGT',
 'GTG',
 'TGG',
 'GGC',
 'GCT',
 'CTA',
 'TAA',
 'AAT',
 'ATG',
 'TGA',
 'GAA',
 'AAG',
 'AGT',
 'GTG',
 'TGA',
 'GAG',
 'AGC',
 'GCG',
 'CGG',
 'GGG',
 'GGA',
 'GAT',
 'ATG',
 'TGA',
 'GAA',
 'AAT',
 'ATA',
 'TAT',
 'ATC',
 'TCG',
 'CGT',
 'GTT',
 'TTT',
 'TTT',
 'TTC',
 'TCC',
 'CCA',
 'CAA',
 'AAA',
 'AAG',
 'AGC',
 'GCG',


In [61]:
text_field.build_vocab(train)


AttributeError: 'Example' object has no attribute 'seq'

In [54]:
train = torchtext.legacy.data.TabularDataset('train.tsv', "tsv", fields)
test = torchtext.legacy.data.TabularDataset('test.tsv', "tsv", fields)
valid = torchtext.legacy.data.TabularDataset('valid.tsv', "tsv", fields)

text_field.build_vocab(train)

torchtext_train_dataloader, torchtext_valid_dataloader = torchtext.legacy.data.BucketIterator.splits((train, valid),
                                           batch_size=16,
                                           sort_key=lambda x: len(x.seq), 
                                           sort_within_batch=True,        
                                           repeat=False)

AttributeError: 'Example' object has no attribute 'seq'

In [53]:
#import csv

#with open('gram_przewidywanie/genes_with_labels.tsv', 'wt') as file:
#    tsv_writer = csv.writer(file, delimiter='\t')
#    for elem in data:
#        tsv_writer.writerow(elem)


def tokenizator(seq):
    reprezentacja = []
    for i in range(len(seq)-2):
        reprezentacja.append(seq[i:(i+3)])
    return reprezentacja

text_field = torchtext.legacy.data.Field(sequential=True,      
                                  tokenize=tokenizator, 
                                  include_lengths=True, 
                                  batch_first=True,
                                  use_vocab=True)      

label_field = torchtext.legacy.data.Field(sequential=False,    
                                   use_vocab=False,     
                                   is_target=True,      
                                   batch_first=True,
                                   preprocessing=lambda x: int(x == 'e')) 

fields = [('label', label_field), ('seq', text_field)]

In [15]:
train = torchtext.legacy.data.TabularDataset('gram_przewidywanie/genes_with_labels.tsv', "tsv", fields)

In [16]:
train

In [19]:
model_3nuc.forward(torchtext_train_dataloader)

AttributeError: 'tuple' object has no attribute 'unsqueeze'

In [36]:
torchtext_train_dataloader, torchtext_valid_dataloader = torchtext.legacy.data.BucketIterator.splits((train, train),
                                           batch_size=16)#,
                                           #sort_key=lambda x: len(x.seq), 
                                           #sort_within_batch=True,        
                                           #repeat=False)

In [35]:
torchtext_train_dataloader.seq

AttributeError: 'BucketIterator' object has no attribute 'seq'

In [37]:
torchtext_train_dataloader.create_batches()
for num, batch in enumerate(torchtext_train_dataloader): #.batches):
            
    #print(torchtext_train_dataloader[0])
            # Put all example.text of batch in single array.
    batch_text = list(batch)[0]
    batch_label = list(batch)[1]
    x = torch.tensor(batch_text[0])
    pred = model_3nuc(x)
    print(pred)

AttributeError: 'Example' object has no attribute 'label'

In [25]:
model_3nuc.parameters()

<generator object Module.parameters at 0x000001ED69803190>